# 信賴區間

## RF + Bootstrap CI

In [5]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from tqdm import tqdm

# === 載入原始資料 ===
file_path = '/Users/yuchingchen/Documents/專題/cleaned_data/long_flight.csv'
data = pd.read_csv(file_path)

# 分離經濟艙與非經濟艙的數據
economy_data = data[data["艙等（主航段）"] == "經濟艙"]
business_data = data[data["艙等（主航段）"] == "商務艙"]

# One-hot encoding：針對多個分類欄位進行處理
categorical_cols = ['星期', '出發時段', '出發機場代號', '抵達時段', '抵達機場代號',
                    '航空公司組合', '航空聯盟', '停靠站數量', '是否過夜', '是否為平日', 
                    '機型分類', '假期', 'Region', '飛行時間兩段分類']
economy_data = pd.get_dummies(economy_data, columns=categorical_cols, drop_first=True)
business_data = pd.get_dummies(business_data, columns=categorical_cols, drop_first=True)

# 數值型特徵標準化
num_cols = ['停留時間_分鐘', '飛行時間_分鐘', '實際飛行時間_分鐘', '經濟指標', '機場指標', 'competing_flights']
scaler = StandardScaler()
economy_data[num_cols] = scaler.fit_transform(economy_data[num_cols])
business_data[num_cols] = scaler.transform(business_data[num_cols])

# 確保兩個數據集擁有相同的特徵欄位
common_cols = list(set(economy_data.columns) & set(business_data.columns))
economy_data = economy_data[common_cols]
business_data = business_data[common_cols]

# 選取建模特徵：先挑選 dummy 欄位，再加上數值型特徵
target_col = '最低價格_log'
target_keywords = ['出發時段_', '出發機場代號_', '抵達時段_', '航空公司組合_', '航空聯盟_', '停靠站數量_', 
                   '機型分類_', '假期_', '飛行時間兩段分類_']
economy_data_dummy_cols = [col for col in economy_data.columns if any(keyword in col for keyword in target_keywords)]
business_data_dummy_cols = [col for col in business_data.columns if any(keyword in col for keyword in target_keywords)]
eco_features = economy_data_dummy_cols + ['停留時間_分鐘', '實際飛行時間_分鐘', '經濟指標', '機場指標', 'competing_flights']
biz_features = business_data_dummy_cols + ['停留時間_分鐘', '實際飛行時間_分鐘', '經濟指標', '機場指標', 'competing_flights']

# Bootstrap 預測區間（95% + 99%）與信心評分
def bootstrap_prediction_ci_rf(X, y_log, label="economy", n_iterations=100):
    X_train, X_test, y_train_log, y_test_log = train_test_split(X, y_log, test_size=0.3, random_state=123)
    y_test = np.exp(y_test_log.values)
    all_log_preds = []

    for i in tqdm(range(n_iterations), desc=f"Bootstrap RF ({label})"):
        X_resample, y_resample = resample(X_train, y_train_log, replace=True, random_state=100+i)
        model = RandomForestRegressor(n_estimators=100, n_jobs=-1, random_state=100+i)
        model.fit(X_resample, y_resample)
        pred_log = model.predict(X_test)
        all_log_preds.append(pred_log)

    all_log_preds = np.array(all_log_preds)
    mean_log_pred = np.mean(all_log_preds, axis=0)
    mean_pred = np.exp(mean_log_pred)

    # CI in log space
    ci_95_lower_log = np.percentile(all_log_preds, 2.5, axis=0)
    ci_95_upper_log = np.percentile(all_log_preds, 97.5, axis=0)
    ci_99_lower_log = np.percentile(all_log_preds, 0.5, axis=0)
    ci_99_upper_log = np.percentile(all_log_preds, 99.5, axis=0)

    # exp back
    ci_95_lower = np.exp(ci_95_lower_log)
    ci_95_upper = np.exp(ci_95_upper_log)
    ci_99_lower = np.exp(ci_99_lower_log)
    ci_99_upper = np.exp(ci_99_upper_log)

    # 是否落在區間
    cover_95 = (y_test >= ci_95_lower) & (y_test <= ci_95_upper)
    cover_99 = (y_test >= ci_99_lower) & (y_test <= ci_99_upper)

    # 信心評估：與 CI 中心距離佔區間比例
    ci_center = (ci_95_upper + ci_95_lower) / 2
    ci_width = ci_95_upper - ci_95_lower
    confidence_score = 1 - np.abs(y_test - ci_center) / (ci_width + 1e-8)
    confidence_score = np.clip(confidence_score, 0, 1)

    # 統計指標
    mse = mean_squared_error(y_test, mean_pred)
    r2 = r2_score(y_test, mean_pred)
    cover_95_ratio = cover_95.mean()
    cover_99_ratio = cover_99.mean()

    print(f"\n📊 [{label}] RF 統計指標：")
    print(f"   MSE         = {mse:.2f}")
    print(f"   R^2         = {r2:.4f}")
    print(f"   95% 覆蓋率   = {cover_95_ratio:.2%}")
    print(f"   99% 覆蓋率   = {cover_99_ratio:.2%}\n")

    df_result = pd.DataFrame({
        "實際價格": y_test,
        "預測值": mean_pred,
        "CI95下限": ci_95_lower,
        "CI95上限": ci_95_upper,
        "CI99下限": ci_99_lower,
        "CI99上限": ci_99_upper,
        "CI95長度": ci_95_upper - ci_95_lower,
        "CI99長度": ci_99_upper - ci_99_lower,
        "是否落在CI95": cover_95.astype(int),
        "是否落在CI99": cover_99.astype(int),
        "信心分數": confidence_score
    })

# 執行經濟艙與商務艙 RF 模型
bootstrap_prediction_ci_rf(economy_data[eco_features], economy_data[target_col], label="economy")
bootstrap_prediction_ci_rf(business_data[biz_features], business_data[target_col], label="business")

Bootstrap RF (economy): 100%|██████████| 100/100 [04:30<00:00,  2.70s/it]



📊 [economy] RF 統計指標：
   MSE         = 41595028.71
   R^2         = 0.9506
   95% 覆蓋率   = 48.31%
   99% 覆蓋率   = 58.13%



Bootstrap RF (business): 100%|██████████| 100/100 [03:11<00:00,  1.91s/it]


📊 [business] RF 統計指標：
   MSE         = 210566918.90
   R^2         = 0.9108
   95% 覆蓋率   = 50.27%
   99% 覆蓋率   = 60.21%



## XGB + Bootstrap CI

In [4]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from tqdm import tqdm

# === 載入原始資料 ===
file_path = '/Users/yuchingchen/Documents/專題/cleaned_data/long_flight.csv'
data = pd.read_csv(file_path)

# 分離經濟艙與非經濟艙的數據
economy_data = data[data["艙等（主航段）"] == "經濟艙"]
business_data = data[data["艙等（主航段）"] == "商務艙"]

# One-hot encoding：針對多個分類欄位進行處理
categorical_cols = ['星期', '出發時段', '出發機場代號', '抵達時段', '抵達機場代號',
                    '航空公司組合', '航空聯盟', '停靠站數量', '是否過夜', '是否為平日', 
                    '機型分類', '假期', 'Region', '飛行時間兩段分類']
economy_data = pd.get_dummies(economy_data, columns=categorical_cols, drop_first=True)
business_data = pd.get_dummies(business_data, columns=categorical_cols, drop_first=True)

# 數值型特徵標準化
num_cols = ['停留時間_分鐘', '飛行時間_分鐘', '實際飛行時間_分鐘', '經濟指標', '機場指標', 'competing_flights']
scaler = StandardScaler()
economy_data[num_cols] = scaler.fit_transform(economy_data[num_cols])
business_data[num_cols] = scaler.transform(business_data[num_cols])

# 確保兩個數據集擁有相同的特徵欄位
common_cols = list(set(economy_data.columns) & set(business_data.columns))
economy_data = economy_data[common_cols]
business_data = business_data[common_cols]

# 選取建模特徵：先挑選 dummy 欄位，再加上數值型特徵
target_col = '最低價格_log'
target_keywords = ['出發時段_', '出發機場代號_', '抵達時段_', '航空公司組合_', '航空聯盟_', '停靠站數量_', 
                   '機型分類_', '假期_', '飛行時間兩段分類_']
economy_data_dummy_cols = [col for col in economy_data.columns if any(keyword in col for keyword in target_keywords)]
business_data_dummy_cols = [col for col in business_data.columns if any(keyword in col for keyword in target_keywords)]
eco_features = economy_data_dummy_cols + ['停留時間_分鐘', '實際飛行時間_分鐘', '經濟指標', '機場指標', 'competing_flights']
biz_features = business_data_dummy_cols + ['停留時間_分鐘', '實際飛行時間_分鐘', '經濟指標', '機場指標', 'competing_flights']

# 修改後的 bootstrap 函式：利用 X_test 的 index 產生結果 DataFrame
def bootstrap_prediction_ci(X, y_log, label="economy", n_iterations=500):
    X_train, X_test, y_train_log, y_test_log = train_test_split(X, y_log, test_size=0.3, random_state=123)
    y_test = np.exp(y_test_log.values)
    all_log_preds = []

    for i in tqdm(range(n_iterations), desc=f"Bootstrap ({label})"):
        X_resample, y_resample = resample(X_train, y_train_log, replace=True, random_state=100+i)
        model = XGBRegressor(n_estimators=100, n_jobs=-1, random_state=100+i)
        model.fit(X_resample, y_resample)
        pred_log = model.predict(X_test)
        all_log_preds.append(pred_log)

    all_log_preds = np.array(all_log_preds)
    mean_log_pred = np.mean(all_log_preds, axis=0)
    mean_pred = np.exp(mean_log_pred)

    # CI 在 log 空間計算
    ci_95_lower_log = np.percentile(all_log_preds, 2.5, axis=0)
    ci_95_upper_log = np.percentile(all_log_preds, 97.5, axis=0)
    ci_99_lower_log = np.percentile(all_log_preds, 0.5, axis=0)
    ci_99_upper_log = np.percentile(all_log_preds, 99.5, axis=0)

    # 轉回原始空間
    ci_95_lower = np.exp(ci_95_lower_log)
    ci_95_upper = np.exp(ci_95_upper_log)
    ci_99_lower = np.exp(ci_99_lower_log)
    ci_99_upper = np.exp(ci_99_upper_log)

    # 是否落在區間
    cover_95 = (y_test >= ci_95_lower) & (y_test <= ci_95_upper)
    cover_99 = (y_test >= ci_99_lower) & (y_test <= ci_99_upper)

    # 信心評估：距離 CI 中心的比例
    ci_center = (ci_95_upper + ci_95_lower) / 2
    ci_width = ci_95_upper - ci_95_lower
    confidence_score = 1 - np.abs(y_test - ci_center) / (ci_width + 1e-8)
    confidence_score = np.clip(confidence_score, 0, 1)

    # 統計指標
    mse = mean_squared_error(y_test, mean_pred)
    r2 = r2_score(y_test, mean_pred)
    cover_95_ratio = cover_95.mean()
    cover_99_ratio = cover_99.mean()

    print(f"\n📊 [{label}] 統計指標：")
    print(f"   MSE         = {mse:.2f}")
    print(f"   R^2         = {r2:.4f}")
    print(f"   95% 覆蓋率   = {cover_95_ratio:.2%}")
    print(f"   99% 覆蓋率   = {cover_99_ratio:.2%}\n")

    # 將結果與 X_test 的 index 一起存入 DataFrame
    df_result = pd.DataFrame({
        "實際價格": y_test,
        "預測值": mean_pred,
        "CI95下限": ci_95_lower,
        "CI95上限": ci_95_upper,
        "CI99下限": ci_99_lower,
        "CI99上限": ci_99_upper,
        "CI95長度": ci_95_upper - ci_95_lower,
        "CI99長度": ci_99_upper - ci_99_lower,
        "是否落在CI95": cover_95.astype(int),
        "是否落在CI99": cover_99.astype(int),
        "信心分數": confidence_score
    }, index=X_test.index)
    print(f"✅ [{label}] Bootstrap CI 結果產生完成！")
    return df_result

# 分別針對經濟艙與商務艙取得預測結果（注意：這裡只包含測試集部分）
eco_result = bootstrap_prediction_ci(economy_data[eco_features], economy_data[target_col], label="economy")
biz_result = bootstrap_prediction_ci(business_data[biz_features], business_data[target_col], label="business")

# 將經濟艙與商務艙的測試集結果合併
test_result = pd.concat([eco_result, biz_result]).sort_index()

# 如果希望同時保留原始資料中的其他資訊，可以利用 test_result 的 index 從原始資料中取出對應列
# 這裡假設原始資料的 index 與 one-hot 過後的資料 index 是一致的
data_test_with_ci = data.loc[test_result.index].copy()
data_test_with_ci = data_test_with_ci.join(test_result, how='left')

# 存檔只包含測試集與預測區間的結果
output_path = '/Users/yuchingchen/Documents/專題/ci/ci_data/long_xgb_with_ci.csv'
data_test_with_ci.to_csv(output_path, index=False)
print(f"✅ 測試集（有預測區間）結果已存檔：{output_path}")

Bootstrap (economy): 100%|██████████| 500/500 [02:31<00:00,  3.30it/s]



📊 [economy] 統計指標：
   MSE         = 50305390.93
   R^2         = 0.9402
   95% 覆蓋率   = 55.67%
   99% 覆蓋率   = 67.87%

✅ [economy] Bootstrap CI 結果產生完成！


Bootstrap (business): 100%|██████████| 500/500 [02:34<00:00,  3.23it/s]



📊 [business] 統計指標：
   MSE         = 239298881.63
   R^2         = 0.8986
   95% 覆蓋率   = 61.92%
   99% 覆蓋率   = 73.83%

✅ [business] Bootstrap CI 結果產生完成！
✅ 測試集（有預測區間）結果已存檔：/Users/yuchingchen/Documents/專題/ci/ci_data/long_xgb_with_ci.csv


In [13]:
# 讀入測試集資料
data = pd.read_csv("/Users/yuchingchen/Documents/專題/ci/ci_data/long_xgb_with_ci.csv")

# 分離經濟艙與商務艙
economy_data = data[data["艙等（主航段）"] == "經濟艙"]
business_data = data[data["艙等（主航段）"] == "商務艙"]

# 顯示 CI95長度 與 CI99長度 的平均
mean_ci_95_length_economy = economy_data["CI95長度"].mean()
mean_ci_99_length_economy = economy_data["CI99長度"].mean()
print(f"經濟艙的平均 CI95 長度: {mean_ci_95_length_economy:.2f}")
print(f"經濟艙的平均 CI99 長度: {mean_ci_99_length_economy:.2f}")
mean_ci_95_length_business = business_data["CI95長度"].mean()
mean_ci_99_length_business = business_data["CI99長度"].mean()
print(f"商務艙的平均 CI95 長度: {mean_ci_95_length_business:.2f}")
print(f"商務艙的平均 CI99 長度: {mean_ci_99_length_business:.2f}")

經濟艙的平均 CI95 長度: 5745.91
經濟艙的平均 CI99 長度: 7857.87
商務艙的平均 CI95 長度: 14981.74
商務艙的平均 CI99 長度: 20582.57
